# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,goure,13.9835,10.2704,31.96,36,100,2.56,NE,1689879763
1,1,blackmans bay,-43.0167,147.3167,4.74,89,12,2.75,AU,1689879763
2,2,bethel,41.3712,-73.4140,30.41,61,40,2.57,US,1689879764
3,3,port mathurin,-19.6833,63.4167,23.28,72,5,8.81,MU,1689879641
4,4,san patricio,28.0170,-97.5169,37.87,41,5,7.29,US,1689879765


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size="Humidity",
    alpha=.5,
    color = "Country")

# Display the map
map1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<22,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>19,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.6,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud
# Drop any rows with null values
city_ideal = city_cloud.dropna()
# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
134,134,mugia,43.1041,-9.2179,21.62,79,0,2.26,ES,1689879837
187,187,fisterra,42.9049,-9.2629,21.53,82,0,3.37,ES,1689879772
203,203,viana do castelo,41.6932,-8.8329,19.16,82,0,3.91,PT,1689879873
256,256,zaysan,47.4667,84.8667,20.99,47,0,3.40,KZ,1689879898
303,303,urzhar,47.0930,81.6294,21.67,49,0,2.18,KZ,1689879658
309,309,sinop,41.7711,34.8709,19.14,80,0,1.35,TR,1689879926


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
134,mugia,ES,43.1041,-9.2179,79,
187,fisterra,ES,42.9049,-9.2629,82,
203,viana do castelo,PT,41.6932,-8.8329,82,
256,zaysan,KZ,47.4667,84.8667,47,
303,urzhar,KZ,47.0930,81.6294,49,
309,sinop,TR,41.7711,34.8709,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 11000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mugia - nearest hotel: No hotel found
fisterra - nearest hotel: No hotel found
viana do castelo - nearest hotel: No hotel found
zaysan - nearest hotel: No hotel found
urzhar - nearest hotel: No hotel found
sinop - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
134,mugia,ES,43.1041,-9.2179,79,No hotel found
187,fisterra,ES,42.9049,-9.2629,82,No hotel found
203,viana do castelo,PT,41.6932,-8.8329,82,No hotel found
256,zaysan,KZ,47.4667,84.8667,47,No hotel found
303,urzhar,KZ,47.0930,81.6294,49,No hotel found
309,sinop,TR,41.7711,34.8709,80,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)